### Setup

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import re
import os
from getpass import getpass

In [2]:
with open('./students-project-fall2023-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········




```
# Als Code formatiert
```

Install required packages and dependencies. Ignore the warnings.

In [3]:
!pip install loguru
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 862.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.5/512.5 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

Import packages

In [4]:
import json
import time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

In [5]:
leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

[
    {
        "name": "worker-07-lease",
        "id": "6e8d7ca8-4488-4712-a1b8-a3eb205264b9",
        "reservation_id": "38c7f405-ad3a-4f5d-a232-72338301d3d9",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:16:00.000000"
    },
    {
        "name": "worker-08-lease",
        "id": "7a10d77b-6f58-418e-8545-e91dc708a7f0",
        "reservation_id": "dda72c1c-d5be-48b2-a916-c42f83a94019",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:17:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "80375624-08ad-4542-9e10-43368dcce2ac",
        "reservation_id": "5986aa6b-b034-4728-b526-294eb2e88ee4",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:14:00.000000"
    },
    {
        "name": "adv-03-lease",
        "id": "efb168f7-aaf4-4569-964e-97114f73a49c",
        "reservation_id": "8da6e1b5-6d8a-4898-9ac3-6063999f6cb9",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:15:00.000000"
    }
]


In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [6]:
# ep5g reservation
ep5g_lease = reserve(
    { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": "100", "duration": { "days":7, "hours":0 } }
)

# advantech router reservation
adv3_lease = reserve(
    { "type":"network", "name": "adv-03", "net_name": "adv-03", "segment_id": "133", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker7_lease = reserve(
    { "type":"device", "name":"worker-07", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker8_lease = reserve(
    { "type":"device", "name":"worker-08", "duration": { "days":7, "hours":0 } }
)

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2023-11-12 10:14:13.543 | INFO     | chi.expeca:reserve:167 - reserving ep5g
2023-11-12 10:14:16.070 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id 80375624-08ad-4542-9e10-43368dcce2ac to become "ACTIVE"
2023-11-12 10:14:21.236 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 80375624-08ad-4542-9e10-43368dcce2ac is PENDING.
2023-11-12 10:14:26.403 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 80375624-08ad-4542-9e10-43368dcce2ac is PENDING.
2023-11-12 10:14:31.564 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 80375624-08ad-4542-9e10-43368dcce2ac is PENDING.
2023-11-12 10:14:36.732 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 80375624-08ad-4542-9e10-43368dcce2ac is PENDING.
2023-11-12 10:14:41.897 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 80375624-08ad-4542-9e10-43368dcce2ac is P

[
    {
        "name": "worker-07-lease",
        "id": "6e8d7ca8-4488-4712-a1b8-a3eb205264b9",
        "reservation_id": "38c7f405-ad3a-4f5d-a232-72338301d3d9",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:16:00.000000"
    },
    {
        "name": "worker-08-lease",
        "id": "7a10d77b-6f58-418e-8545-e91dc708a7f0",
        "reservation_id": "dda72c1c-d5be-48b2-a916-c42f83a94019",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:17:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "80375624-08ad-4542-9e10-43368dcce2ac",
        "reservation_id": "5986aa6b-b034-4728-b526-294eb2e88ee4",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:14:00.000000"
    },
    {
        "name": "adv-03-lease",
        "id": "efb168f7-aaf4-4569-964e-97114f73a49c",
        "reservation_id": "8da6e1b5-6d8a-4898-9ac3-6063999f6cb9",
        "status": "ACTIVE",
        "end_date": "2023-11-19T10:15:00.000000"
    }
]


In [6]:
leaseslist = list_reservations(brief=True)

adv3_lease = [lease for lease in leaseslist if lease["name"]=="adv-03-lease"][0]
worker7_lease = [lease for lease in leaseslist if lease["name"]=="worker-07-lease"][0]
worker8_lease = [lease for lease in leaseslist if lease["name"]=="worker-08-lease"][0]

ep5g_lease = [lease for lease in leaseslist if lease["name"]=="ep5g-lease"][0]
worker7_reservation_id = worker7_lease["reservation_id"]
worker8_reservation_id = worker8_lease["reservation_id"]

print(adv3_lease,ep5g_lease, worker7_reservation_id, worker8_reservation_id)

{'name': 'adv-03-lease', 'id': 'efb168f7-aaf4-4569-964e-97114f73a49c', 'reservation_id': '8da6e1b5-6d8a-4898-9ac3-6063999f6cb9', 'status': 'ACTIVE', 'end_date': '2023-11-19T10:15:00.000000'} {'name': 'ep5g-lease', 'id': '80375624-08ad-4542-9e10-43368dcce2ac', 'reservation_id': '5986aa6b-b034-4728-b526-294eb2e88ee4', 'status': 'ACTIVE', 'end_date': '2023-11-19T10:14:00.000000'} 38c7f405-ad3a-4f5d-a232-72338301d3d9 dda72c1c-d5be-48b2-a916-c42f83a94019


In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).
It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [8]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

2023-11-12 10:18:48.829 | SUCCESS  | __main__:<cell line: 4>:4 - edge-net is created.
2023-11-12 10:18:51.114 | SUCCESS  | __main__:<cell line: 8>:8 - ep5g-vip-router router is created.
2023-11-12 10:18:51.117 | INFO     | __main__:<cell line: 9>:9 - {
    "id": "74fe06df-3255-4fff-a894-484f946073d3",
    "name": "ep5g-vip-router",
    "tenant_id": "ddf790cc1bb64e0387eff22c1ab19f94",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.126"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2023-11-12T10:18:49Z",
    "updated_at": "2023-11-12T10:18:50Z",
    "revision_num

Now the network is ready to run the workloads. We start by running the edge-node perf-meas container.

In [9]:
edgenet = chi.network.get_network("edge-net")
chi.container.create_container(
    name = "edge-node",
    image = "gourav4871/perf-meas-full",
    reservation_id = worker8_reservation_id,
    environment = {"SERVER_DIR":"/mnt/volume/"},
    mounts = [
        {'source': 'edge-volume', 'destination': '/mnt/volume/'}
    ],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.70.70.3/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active("edge-node")
logger.success("created edge-node container.")

2023-11-12 10:20:25.494 | SUCCESS  | __main__:<cell line: 20>:20 - created edge-node container.


Next, we run the end-node perf-meas container.

In [10]:
advnet = chi.network.get_network("adv-03-net")
chi.container.create_container(
    name = "end-node",
    image = "gourav4871/perf-meas-full",
    reservation_id = worker7_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active("end-node")
logger.success("created end-node container.")

2023-11-12 10:21:18.873 | SUCCESS  | __main__:<cell line: 17>:17 - created end-node container.


Run uplink bandwidth test

In [12]:
command = "iperf3 -c 10.70.70.3 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 14:47:41.895 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


Run downlink bandwidth test

In [23]:
command = "iperf3 -c 172.16.0.96 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 12:33:06.239 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


### Uplink measurements

In [13]:
# set session parameters
end_name = "endnode01"
edge_name = "edge"
nt_dev = "adv03"
exp_name = "prova"
client_ip = "172.16.0.40"
server_ip = "10.70.70.3"
sleep_dur = 10
tripm = "oneway"
total_samples = 1e6
interval = "10.5" # in ms
offset = "0" # ms
length = 23520 # in B
mult = 4 # multiple of packets
duration = 10 # in seconds
total_session_time = interval # in seconds
itnum = 2 #math.ceil(total_session_time/duration)
cl_port = "55500"
nt_sleep = "300" # in ms

In [14]:
# run measurements at end node
command = f"DOWNLINK=false EDGE_NAME={edge_name} END_NAME={end_name} NT_DEV={nt_dev} EXP_NAME={exp_name} SERVER_IP={server_ip} CLIENT_IP={client_ip} ITNUM={itnum} SLEEP_DUR={sleep_dur} TRIPM={tripm} INTERVAL={interval}ms INTERVAL_OFFSET={offset}ms LENGTH={length} MULT={mult} DUR={duration}s CL_PORT={cl_port} NT_SLEEP={nt_sleep}ms /tmp/measure-upload.sh > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 14:49:33.145 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [13]:
 # make parquet
dev_name = "endnode01"
trip = "uplink"

command = f"python3 /tmp/parquets-from-folders.py /mnt/volume/{exp_name}/results /mnt/volume/{exp_name}/client /mnt/volume/{exp_name}/edge/server /mnt/volume/{exp_name}/networkinfo trip={trip} > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 10:40:39.976 | INFO     | __main__:<cell line: 10>:10 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [14]:
# upload parquet files to containers
command = "for f in /mnt/volume/{}/results/*.parquet; do AUTH_SERVER=testbed.expeca.proj.kth.se AUTH_PROJECT_NAME=students-project-fall2023 AUTH_USERNAME=ata AUTH_PASSWORD=TEOKIDNCP3UrmIEqu6tLwSXAU2zRZXM0xlpwtQGo python3 /tmp/upload-files.py $f {}; done".format(exp_name,exp_name)
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 10:43:28.589 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


### Downlink Measurement

In [10]:
# set session parameters
end_name = "endnode01"
edge_name = "edge"
nt_dev = "adv03"
exp_name = "session19-DL"
server_ip = "172.16.0.40"
client_ip = "10.70.70.3"
sleep_dur = 30
tripm = "oneway"
total_samples = 1e6
interval = 5500 # in us
offset = "0" # ms
length = 22220 # in B
mult = 8 # multiple of packets
duration = 1810 # in seconds
total_session_time = interval # in seconds
itnum = 1 #math.ceil(total_session_time/duration)
cl_port = "55500"
nt_sleep = "300" # in ms

In [11]:
 ###### run measurements at edge node #####
command = f"DOWNLINK=true EDGE_NAME={edge_name} END_NAME={end_name} NT_DEV={nt_dev} EXP_NAME={exp_name} SERVER_IP={server_ip} CLIENT_IP={client_ip} ITNUM={itnum} SLEEP_DUR={sleep_dur} TRIPM={tripm} INTERVAL={interval}us INTERVAL_OFFSET={offset}ms LENGTH={length} MULT={mult} DUR={duration}s CL_PORT={cl_port} NT_SLEEP={nt_sleep}ms /tmp/measure-upload.sh > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 14:43:39.149 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [8]:
dev_name = "endnode01"
trip = "downlink"
 # make parquet
command = f"python3 /tmp/parquets-from-folders.py /mnt/volume/{exp_name}/results /mnt/volume/{exp_name}/edge/client /mnt/volume/{exp_name}/{dev_name}/server /mnt/volume/{exp_name}/{dev_name}/networkinfo trip={trip} > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 14:30:27.106 | INFO     | __main__:<cell line: 9>:9 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [9]:
# upload parquet files to containers
command = "for f in /mnt/volume/{}/results/*.parquet; do AUTH_SERVER=testbed.expeca.proj.kth.se AUTH_PROJECT_NAME=students-project-fall2023 AUTH_USERNAME=ata AUTH_PASSWORD=TEOKIDNCP3UrmIEqu6tLwSXAU2zRZXM0xlpwtQGo python3 /tmp/upload-files.py $f {}; done".format(exp_name,exp_name)
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-11-14 14:33:55.911 | INFO     | __main__:<cell line: 7>:7 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


### Reset Setup

CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

In [17]:
status = get_container_status("edge-node")
if status:
    chi.container.destroy_container("edge-node")
    wait_until_container_removed("edge-node")


status = get_container_status("end-node")
if status:
    chi.container.destroy_container("end-node")
    wait_until_container_removed("end-node")

logger.info(f"stopped and removed all containers")

# find the router again
router = None
try:
    router = chi.network.get_router("ep5g-vip-router")
except Exception as ex:
    logger.info(f"could not find ep5g-vip-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success(f"removed all routers from router")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success(f"removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success(f"deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("edge-net")
except Exception as ex:
    logger.info(f"could not find edge-net.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success(f"deleted the edge-net")

# remove the leases
# unreserve_byid(ep5g_lease["id"])
# unreserve_byid(adv_lease["id"])
# unreserve_byid(worker_lease["id"])

2023-11-12 10:09:29.738 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for edge-node container to be removed
2023-11-12 10:09:35.052 | INFO     | chi.expeca:wait_until_container_removed:37 - container edge-node is in Running state.
2023-11-12 10:09:40.370 | INFO     | chi.expeca:wait_until_container_removed:37 - container edge-node is in None state.
2023-11-12 10:09:41.000 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for end-node container to be removed
2023-11-12 10:09:46.300 | INFO     | chi.expeca:wait_until_container_removed:37 - container end-node is in Running state.
2023-11-12 10:09:51.605 | INFO     | chi.expeca:wait_until_container_removed:37 - container end-node is in None state.
2023-11-12 10:09:51.608 | INFO     | __main__:<cell line: 12>:12 - stopped and removed all containers
2023-11-12 10:09:55.554 | SUCCESS  | __main__:<cell line: 21>:24 - removed all routers from router
2023-11-12 10:09:55.884 | INFO     | _

In [18]:
unreserve_byid(ep5g_lease["id"])
unreserve_byid(adv3_lease["id"])
unreserve_byid(worker7_lease["id"])
unreserve_byid(worker8_lease["id"])


2023-11-12 10:10:15.764 | INFO     | chi.expeca:remove_lease:84 - Removing ep5g-lease reservation with id 01167fc2-b7d8-4099-985b-72990e6c16c6.
2023-11-12 10:10:36.346 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id 01167fc2-b7d8-4099-985b-72990e6c16c6 to become "None"
2023-11-12 10:10:41.623 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id 01167fc2-b7d8-4099-985b-72990e6c16c6 is None.
2023-11-12 10:10:41.625 | SUCCESS  | chi.expeca:try_to_remove:95 - done
2023-11-12 10:10:41.896 | INFO     | chi.expeca:remove_lease:84 - Removing adv-03-lease reservation with id 1f501e89-5afb-4278-b4e1-1f23bb95e3f4.
2023-11-12 10:10:56.018 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for adv-03-lease with id 1f501e89-5afb-4278-b4e1-1f23bb95e3f4 to become "None"
2023-11-12 10:11:01.290 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-03-lease with id 1f501e89-5afb-4278-b4e1-1f23bb95e3f4

In [19]:
ep5g_lease = [lease for lease in leaseslist if lease["name"]=="ep5g-lease"][0]
unreserve_byid(ep5g_lease["id"])
print(ep5g_lease)

2023-11-12 10:11:20.747 | ERROR    | chi.expeca:unreserve_byid:164 - no reservation found with id 01167fc2-b7d8-4099-985b-72990e6c16c6


{'name': 'ep5g-lease', 'id': '01167fc2-b7d8-4099-985b-72990e6c16c6', 'reservation_id': 'dfc027d9-750d-453e-8c19-104fd60ba97e', 'status': 'ACTIVE', 'end_date': '2023-11-17T13:36:00.000000'}
